In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('adult.csv')
data

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [24]:
#Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
#Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

# попробуем выбрать какие-нибудь признаки
selectedColumns = data[[ 'age', 'workclass', 'educational-num', 'marital-status', 'occupation', 
                         'relationship', 'race', 'gender', 'capital-gain', 'capital-loss',
                        'hours-per-week', 'native-country', 'income']]
selectedColumns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   educational-num  48842 non-null  int64 
 3   marital-status   48842 non-null  object
 4   occupation       48842 non-null  object
 5   relationship     48842 non-null  object
 6   race             48842 non-null  object
 7   gender           48842 non-null  object
 8   capital-gain     48842 non-null  int64 
 9   capital-loss     48842 non-null  int64 
 10  hours-per-week   48842 non-null  int64 
 11  native-country   48842 non-null  object
 12  income           48842 non-null  object
dtypes: int64(5), object(8)
memory usage: 4.8+ MB


In [30]:
# проверяем на отсутствие нулевых значений
selectedColumns.any().isna()

age                False
workclass          False
educational-num    False
marital-status     False
occupation         False
relationship       False
race               False
gender             False
capital-gain       False
capital-loss       False
hours-per-week     False
native-country     False
income             False
dtype: bool

In [4]:
# проверяем колонки с нулевыми данными
for key in selectedColumns.keys():
    print(key, selectedColumns[key][selectedColumns[key].isnull()].count())

age 0
workclass 0
educational-num 0
marital-status 0
occupation 0
relationship 0
race 0
gender 0
capital-gain 0
capital-loss 0
hours-per-week 0
native-country 0
income 0


In [5]:
# проверяем колонки с данными = "?"
for key in selectedColumns.keys():
    print(key, selectedColumns[key][selectedColumns[key] == '?'].count())
   

age 0
workclass 2799
educational-num 0
marital-status 0
occupation 2809
relationship 0
race 0
gender 0
capital-gain 0
capital-loss 0
hours-per-week 0
native-country 857
income 0


c:\program files\python38\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [6]:
#удаляем строки, содержащие "?"
selectedColumns = selectedColumns.drop(selectedColumns['workclass'][selectedColumns['workclass'] == '?'].index)
selectedColumns = selectedColumns.drop(selectedColumns['occupation'][selectedColumns['occupation'] == '?'].index)
selectedColumns = selectedColumns.drop(selectedColumns['native-country'][selectedColumns['native-country'] == '?'].index)

In [7]:
# переводим категориальные значения из columns в значения 0 и 1
X = pd.get_dummies(selectedColumns, columns = ['workclass', 'marital-status', 'occupation', 
                         'relationship', 'race', 'gender', 'native-country'])

# столбец income является целевой переменной, удаляем его из X
del X['income']
print(X.shape)
X.head()

(45222, 87)


,age,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,25,7,0,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,38,9,0,0,50,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,28,12,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,44,10,7688,0,40,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,34,6,0,0,30,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# sc = StandardScaler()
# sc.fit_transform(X[['age']])

ct = ColumnTransformer(remainder='passthrough',
    transformers=[("norm1",StandardScaler(), [0,1,2,3,4])])
x_t = ct.fit_transform(X)

pd.DataFrame(x_t)

,0,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,86
0,-1.024983,-1.221559,-0.146733,-0.21878,-0.078120,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,-0.041455,-0.438122,-0.146733,-0.21878,0.754701,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.798015,0.737034,-0.146733,-0.21878,-0.078120,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.412481,-0.046403,0.877467,-0.21878,-0.078120,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.344079,-1.613277,-0.146733,-0.21878,-0.910942,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,-0.873671,0.737034,-0.146733,-0.21878,-0.244684,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45218,0.109857,-0.438122,-0.146733,-0.21878,-0.078120,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45219,1.471665,-0.438122,-0.146733,-0.21878,-0.078120,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
45220,-1.251951,-0.438122,-0.146733,-0.21878,-1.743763,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [9]:
# целевая переменная (столбец income) является категориальной
# переведем значения столбца в числа, оставив один столбец

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [10]:
le.fit(selectedColumns['income'])
print(le.classes_)

# записываем в переменную y преобразованный столбец income

y = pd.Series(data = le.transform( selectedColumns['income']))
y.head()

['<=50K' '>50K']


0    0
1    0
2    1
3    1
4    0
dtype: int32

In [18]:

LR = LogisticRegression(max_iter=5000)
SV = SVC(gamma='auto')

from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.3, random_state=63, stratify=y)
print(LR, SV, sep='\n')

LogisticRegression(max_iter=5000)
SVC(gamma='auto')


In [15]:
LR.fit(ct.fit_transform(Xtrain), ytrain)
#predictions = model1.predict(X)
print(LR.score(ct.transform(Xtrain), ytrain))
print(LR.score(ct.transform(Xtest), ytest))

0.844390387734041
0.8490649481930755


In [19]:
SV.fit(ct.fit_transform(Xtrain), ytrain)
#predictions = model2.predict(X)
print(SV.score(ct.transform(Xtrain), ytrain))
print(SV.score(ct.transform(Xtest), ytest))

0.846233230134159
0.8522239070002527


# Способ через pipeline:

In [21]:
from sklearn.pipeline import make_pipeline
#Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.3, random_state=63, stratify=y)

In [22]:
pipe_model = make_pipeline(ColumnTransformer(remainder='passthrough',
                            transformers=[("norm1", StandardScaler(), [0,1,2,3,4])]),
                          LogisticRegression(max_iter=5000))
pipe_model.fit(Xtrain, ytrain)
print(pipe_model.score(Xtrain, ytrain))
print(pipe_model.score(Xtest, ytest))

0.844390387734041
0.8490649481930755


In [23]:
#from sklearn.svm import SVC 
pipe_model = make_pipeline(ColumnTransformer(remainder='passthrough',
                            transformers=[("norm1", StandardScaler(), [0,1,2,3,4])]),
                          SVC(gamma='auto'))
pipe_model.fit(Xtrain, ytrain)
print(pipe_model.score(Xtrain, ytrain))
print(pipe_model.score(Xtest, ytest))

0.846233230134159
0.8522239070002527
